## 배낭 (Knapsack) 문제

- 가방(Knapsack)에 물건을 여러 개 담을 때 가방에 담은 물건들의 조합의 가치를 극대화 할 수 있는 방법을 찾는 문제
- 다만, 배낭의 허용 용량을 초과하여 아이템을 담을 수 없다는 제약이 있음

In [1]:
import random
import math

class Knapsack:
    def __init__(self, items, capacity):
        self.items = items #아이템
        self.capacity = capacity #용량

    #목적함수를 구현함
    def evaluate(self, solution): # 총 가치 계산
        total_value = 0
        for i in range(len(self.items)):
            if solution[i] == 1: #1이면 아이템이 존재/ 0이면 아이템이 존재하지 않음
                _, value, _ = self.items[i]
                total_value += value
        return total_value

    def measure_weight(self, solution): # 무게 측정
        total_weight = 0
        for i in range(len(self.items)):
            if solution[i] == 1: #아이템들의 총 무게를 측정
                _, _, weight = self.items[i]
                total_weight += weight
        return total_weight

    def is_valid(self, solution): # 아이템 총합 배낭 허용 용량 초과 여부 확인
        return self.measure_weight(solution) <= self.capacity 

    def get_neighbors(self, solution): # (가능한) 이웃 상태 도출 : 갈 수 있는 다음 상태들의 집합
        all_neighbors = []
        valid_neighbors = []
        for i in range(len(self.items)):
            neighbor = solution[:]
            neighbor[i] = 1 - neighbor[i]  # 물건 포함 여부를 뒤집음 (0 혹은 1) : 물건을 배낭에 넣고 빼고를 구현. 이웃상태를 뽑아냄
            all_neighbors.append(neighbor)
        valid_neighbors = [n for n in all_neighbors if self.is_valid(n)]  # 무게 초과하지 않는 이웃 상태만 도출
        return valid_neighbors

    def random_initial_solution(self): # (가능한) 초기 상태 랜덤 생성
        while True:
            initial_solution = [random.randint(0, 1) for _ in range(len(self.items))]
            if self.is_valid(initial_solution): #허용가능한지 확인
                return initial_solution

In [2]:
# (이름, 가치, 무게) 튜플로 이루어진 아이템 목록
items = [
    ("책", 30, 1),
    ("노트북", 300, 3),
    ("물병", 20, 1),
    ("필통", 10, 1),
    ("간식", 50, 1),
    ("이어폰", 60, 0.5),
    ("공책", 15, 0.5),
    ("전공책", 100, 2)
]


# 배낭 용량
capacity = 5 

# 국소 탐색(local search)

## 언덕 오르기 알고리즘
- 언덕오르기 알고리즘은 하나의 현재 상태를 추적하며, 반복마다 가장 큰 값을 가진 이웃 상태로 이동하는 탐욕적(greedy)인 국소 탐색 기법

```java
Function HILL_CLIMBING(problem)
  current_solution ← problem.initial_solution
  while True
    next_solution ← current_solution의 이웃 상태 중 가장 높은 값을 가진 상태
    if next_solution의 값 > current_solution의 값
      current_solution ← next_solution
  return current_solution
```

In [32]:
import random

def hill_climbing(knapsack):
    # 실습
    current_solution = knapsack.random_initial_solution()
    current_value= knapsack.evaluate(current_solution)
    count =1
    while True:
        neighbors = knapsack.get_neighbors(current_solution)
        evaluated_neighbors = [knapsack.evaluate(n) for n in neighbors]
        print(f"{count}번째 이웃상태: {evaluated_neighbors}")

        next_soultion = None
        for n in neighbors:
            n_value = knapsack.evaluate(n)
            if n_value > current_value:
                next_soultion = n
                current_value = n_value

        if next_soultion is None:
            break

        current_soultion = next_soultion
        count +=1
    return current_solution, current_value

In [33]:
initial_state = Knapsack(items, capacity)
solution, total_value = hill_climbing(initial_state)

# 결과 출력
selected_items = [items[i][0] for i in range(len(items)) if solution[i] == 1]
print(f"결과값: {total_value}, 아이템 목록: {solution}(={selected_items})")

1번째 이웃상태: [50, 380, 100, 90, 30, 140, 95, 180]
2번째 이웃상태: [50, 380, 100, 90, 30, 140, 95, 180]
결과값: 380, 아이템 목록: [1, 0, 0, 0, 1, 0, 0, 0](=['책', '간식'])


## 모의 정련(simulated annealing)
- 모의 정련 알고리즘은 언덕 오르기와 비슷하지만 항상 최상의 이동을 선택하는 대신 금속이나 유리의 정련 과정을 모사한 확률적 요소를 도입한 국소 탐색 기법


```java
Function SIMULATED_ANNEALING(problem)
  current_solution ← problem.initial_solution
  t ← 0
  while True
    T ← schedule(t)
    if T = 0
      return current_solution
    next_state ← current_solution의 이웃 상태 중 무작위 선택
    ΔE ← next_solution의 값 - current_solution의 값
    if ΔE > 0
      current_solution ← next_solution
    else
      current_solution ← next_solution #단, e^(ΔE/T)의 확률로!
    t += 1
  return current_solution
```

- 단, schedule의 경우 냉각 스케쥴링 함수(t가 올라감에 따라 온도를 어떻게 감소시킬지 결정) 필요
- (본 수업에서는 cooling_rate을 도입하여, 시간이 지남에 따라 온도가 선형적으로 감소하도록 만들 계획)

In [34]:
def cooling_schedule(t, initial_temp, cooling_rate):
    temp = initial_temp - cooling_rate * t # 반복마다 선형적으로 온도가 감소
    if temp < 0:
        temp = 0
    return temp

def simulated_annealing(knapsack, initial_temp=1000, cooling_rate=0.95):
    # 실습
    current_solution = knapsack.random_initial_solution()
    current_value= knapsack.evaluate(current_solution)
    count =1
    t=0
    

    while True:
        T = cooling_schedule(t, initial_temp, cooling_rate)
        if T == 0:
            break
        neighbors = knapsack.get_neighbors(current_solution)
        evaluated_neighbors = [knapsack.evaluate(n) for n in neighbors]
        print(f"{count}번째 이웃상태: {evaluated_neighbors}")

        next_solution = random.choice(neighbors)
        next_value = knapsack.evaluate(next_solution)
        delta_e = next_value - current_value
        if delta_e >0 :
            current_solution = next_solution
            current_value = next_value
        else:
            if random.random() < math.exp(delta_e/T):
                current_solution = next_solution
                current_value = next_value

        if next_solution is None:
            break

        current_soultion = next_solution
        count +=1
        if count == 1000:
            break
        t+=1
    return current_solution, current_value
    return current_solution, current_value

In [37]:
initial_state = Knapsack(items, capacity)
solution, total_value = simulated_annealing(initial_state)

# 결과 출력
selected_items = [items[i][0] for i in range(len(items)) if solution[i] == 1]
print(f"결과값: {total_value}, 아이템 목록: {solution}(={selected_items})")

1번째 이웃상태: [80, 90, 120, 160, 50, 125, 210]
2번째 이웃상태: [110, 380, 60, 90, 130, 20, 95, 180]
3번째 이웃상태: [80, 360, 320, 395]
4번째 이웃상태: [95, 375, 335, 380]
5번째 이웃상태: [80, 360, 320, 395]
6번째 이웃상태: [350, 20, 300, 330, 370, 380, 335]
7번째 이웃상태: [330, 0, 320, 310, 350, 360, 315, 400]
8번째 이웃상태: [350, 20, 300, 330, 370, 380, 335]
9번째 이웃상태: [30, 310, 320]
10번째 이웃상태: [60, 330, 10, 20, 80, 90, 45, 130]
11번째 이웃상태: [110, 60, 70, 30, 140, 95, 180]
12번째 이웃상태: [160, 170, 130, 80]
13번째 이웃상태: [110, 60, 70, 30, 140, 95, 180]
14번째 이웃상태: [60, 330, 10, 20, 80, 90, 45, 130]
15번째 이웃상태: [30, 40, 50, 110, 120, 75, 160]
16번째 이웃상태: [60, 330, 10, 20, 80, 90, 45, 130]
17번째 이웃상태: [30, 310, 320]
18번째 이웃상태: [350, 20, 300, 330, 370, 380, 335]
19번째 이웃상태: [70, 350, 320]
20번째 이웃상태: [100, 370, 50, 80, 20, 130, 85, 170]
21번째 이웃상태: [200, 150, 180, 120, 230, 185, 70]
22번째 이웃상태: [150, 100, 130, 170, 180, 135, 20]
23번째 이웃상태: [165, 115, 145, 185, 195, 120, 35]
24번째 이웃상태: [150, 100, 130, 170, 180, 135, 20]
25번째 이웃상태: [160, 110, 120, 1

## 유전 알고리즘(Genetic Algorithm)

- 유전 알고리즘은 진화론의 교차, 돌연변이 등의 아이디어를 차용하여 세대를 거듭하며 다양성을 줄여나가며 더 나은 해를 찾아가는 최적화 기법

1. 초기 모집단
2. 적합도 계산
3. 부모 선택
4. 재생산(교차)
5. 재생산(돌연변이)
6. 다음 세대 채우기

```java
Function GENETIC_ALGORITHM(problem, population_size, mutation_rate, max_generations)
  population ← problem.random_initial_population(population_size)
  best_solution ← null
  best_fitness ← 0
  
  For generation = 1 to max_generations do
    fitness_scores ← [problem.fitness(individual) for individual in population]
    
    max_fitness_in_gen ← max(fitness_scores)
    if max_fitness_in_gen > best_fitness then
      best_fitness ← max_fitness_in_gen
      best_solution ← population[fitness_scores.index(max_fitness_in_gen)]
    
    new_population ← []
    For i = 1 to population_size // 2 do
      parent1 ← problem.select(population, fitness_scores)
      parent2 ← problem.select(population, fitness_scores)
      
      child1, child2 ← problem.crossover(parent1, parent2)
      
      if (random() < mutation_rate) then child1 ← problem.mutate(child1)
      if (random() < mutation_rate) then child2 ← problem.mutate(child2)
      
      new_population.append(child1)
      new_population.append(child2)
    
    population ← new_population
    
  return best_solution, best_fitness
```

- 아래 가방(Knapsack) 문제를 GA 푸는데 적합하도록 수정

In [10]:
import random

class KnapsackGA:
    def __init__(self, items, capacity):
        self.items = items
        self.capacity = capacity
        self.chromosome_size = len(items)  # 염색체 크기 (아이템의 개수)

    # 개체 적합도 계산
    def fitness(self, chromosome):
        total_value = 0
        total_weight = 0 
        for i in range(self.chromosome_size):
            if chromosome[i] == 1:  # 아이템이 선택된 경우
                _, value, weight = self.items[i]
                total_value += value
                total_weight += weight
        if total_weight > self.capacity: # 배낭 용량 초과 시 적합도 0으로 지정해버림
            return 0
        return total_value

    # 초기 모집단 생성
    def random_initial_population(self, population_size):
        population = []
        for _ in range(population_size):
            chromosome = [random.randint(0, 1) for _ in range(self.chromosome_size)] #염색체를 모집단의 크기만큼 랜덤으로 생성 
            population.append(chromosome)
        return population

    # 부모 선택 (룰렛 휠 선택 : 적합도를 기반해 확률적으로 뽑음)
    def select(self, population, fitness_scores):
        total_fitness = sum(fitness_scores) # 전체 적합도 합계
        
        if total_fitness == 0:
            return random.choice(population)  # 적합도가 모두 0일 경우 임의의 부모 반환
        
        pick = random.uniform(0, total_fitness) # 0에서 total_fitness 사이의 무작위 값 선택 (룰렛 휠에서의 선택)

        # 적합도를 누적하여 룰렛 휠에서 선택된 개체 찾기
        current = 0
        for i in range(len(population)):  
            current += fitness_scores[i]  # 적합도를 누적
            if current > pick:  # 무작위 값이 현재 누적된 적합도보다 작으면 개체 선택
                return population[i]
        return population[0] # 기본으로 첫 번째 개체 반환

    # 교차
    def crossover(self, parent1, parent2):
        i = random.randint(1, self.chromosome_size - 1)
        child1 = parent1[:i] + parent2[i:]   #부모특성을 섞음 
        child2 = parent2[:i] + parent1[i:]
        return child1, child2

    # 돌연변이
    def mutate(self, chromosome, mutation_rate): 
        for i in range(self.chromosome_size): #확률에 따라서 유전자를 뒤집을지 말지 결정 
            if random.random() < mutation_rate: #mutation_rate가 높으면 급격히 바뀌기에 낮은값으로 시작 
                chromosome[i] = 1 - chromosome[i]  # 유전자를 0에서 1로, 또는 1에서 0으로 뒤집음


In [15]:
def genetic_algorithm(knapsack_problem, population_size, mutation_rate, max_generations):
    
    population = knapsack_problem.random_initial_population(population_size)
    best_solution = None
    best_fitness = 0

    for generation in range(max_generations):
        fitness_scores = [knapsack_problem.fitness(chromosome) for chromosome in population]
        max_fitness_in_gen = max(fitness_scores)

        if max_fitness_in_gen > best_fitness:
            best_fitness = max_fitness_in_gen
            best_solution = population[fitness_scores.index(max_fitness_in_gen)]
        
        print(f"{generation} 세대 최대 적합도 : {best_fitness}")

        new_population = []

        for _ in range(population_size // 2):
            parent1 = knapsack_problem.select(population, fitness_scores)
            parent2 = knapsack_problem.select(population, fitness_scores)

            child1, child2 = knapsack_problem.crossover(parent1, parent2)
            knapsack_problem.mutate(child1, mutation_rate)
            knapsack_problem.mutate(child2, mutation_rate)

            new_population.append(child1)
            new_population.append(child2)

        population = new_population
    return best_solution, best_fitness


In [5]:
items = [
    ("책", 45, 1.5),
    ("노트북", 350, 2.8),
    ("물병", 35, 0.9),
    ("필통", 20, 0.6),
    ("간식", 75, 0.4),
    ("이어폰", 100, 0.3),
    ("공책", 30, 1.1),
    ("전공책", 150, 2.2),
    ("펜", 8, 0.2),
    ("자", 12, 0.4),
    ("계산기", 90, 0.7),
    ("마커", 25, 0.3),
    ("USB 메모리", 55, 0.1),
    ("파일", 40, 0.8),
    ("가위", 22, 0.5)
]

capacity = 8

In [20]:
knapsack_problem = KnapsackGA(items, capacity)

# 유전 알고리즘 설정
population_size = 10  # 모집단 크기
mutation_rate = 0.05  # 돌연변이 확률
max_generations = 150  # 최대 세대 수

# 유전 알고리즘 실행
best_solution, best_fitness = genetic_algorithm(knapsack_problem, population_size, mutation_rate, max_generations)

print(f"\n최적 해: {best_solution}, 총 가치: {best_fitness}")

0 세대 최대 적합도 : 722
1 세대 최대 적합도 : 797
2 세대 최대 적합도 : 797
3 세대 최대 적합도 : 797
4 세대 최대 적합도 : 797
5 세대 최대 적합도 : 830
6 세대 최대 적합도 : 830
7 세대 최대 적합도 : 830
8 세대 최대 적합도 : 830
9 세대 최대 적합도 : 830
10 세대 최대 적합도 : 830
11 세대 최대 적합도 : 830
12 세대 최대 적합도 : 830
13 세대 최대 적합도 : 830
14 세대 최대 적합도 : 830
15 세대 최대 적합도 : 830
16 세대 최대 적합도 : 830
17 세대 최대 적합도 : 830
18 세대 최대 적합도 : 830
19 세대 최대 적합도 : 830
20 세대 최대 적합도 : 840
21 세대 최대 적합도 : 840
22 세대 최대 적합도 : 840
23 세대 최대 적합도 : 840
24 세대 최대 적합도 : 840
25 세대 최대 적합도 : 840
26 세대 최대 적합도 : 840
27 세대 최대 적합도 : 840
28 세대 최대 적합도 : 840
29 세대 최대 적합도 : 840
30 세대 최대 적합도 : 840
31 세대 최대 적합도 : 840
32 세대 최대 적합도 : 840
33 세대 최대 적합도 : 840
34 세대 최대 적합도 : 840
35 세대 최대 적합도 : 840
36 세대 최대 적합도 : 840
37 세대 최대 적합도 : 840
38 세대 최대 적합도 : 840
39 세대 최대 적합도 : 840
40 세대 최대 적합도 : 840
41 세대 최대 적합도 : 840
42 세대 최대 적합도 : 840
43 세대 최대 적합도 : 840
44 세대 최대 적합도 : 840
45 세대 최대 적합도 : 840
46 세대 최대 적합도 : 840
47 세대 최대 적합도 : 840
48 세대 최대 적합도 : 840
49 세대 최대 적합도 : 840
50 세대 최대 적합도 : 840
51 세대 최대 적합도 : 840
52 세대 최대 적합도 : 840
53 